In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [3]:
#ee.Authenticate()
#geemap.update_package()

Enter verification code: 4/1AY0e-g7BNjK5hG3-aB1Iwcmb2TQPqLH8vRCb58rzlFk73YXAl3pfkBw0OSM

Successfully saved authorization token.


In [ ]:
#converting javascript code to python - also checking to see if same issue occurs with map (weird white coloration):
#js_snippet = """
#var image1 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816'); 
#Map.addLayer(image1, {bands:['B4', 'B3', 'B2']}, 'ColorIR composite');
#"""
#geemap.js_snippet_to_py(js_snippet, add_new_cell = True, import_ee = False, import_geemap = False, show_map = False)

In [77]:
#list of plots in Virginia LTER dataset (not including broad creek)
ee.Initialize()

plot_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-75.372740, 37.903114]), {'site_id': 'Assateague'}), #Assateague
  ee.Feature(ee.Geometry.Point([-75.671577, 37.625273]), {'site_id': 'Bellvue'}), #Bellvue
  ee.Feature(ee.Geometry.Point([-75.876094, 37.395746]), {'site_id': 'Box_Tree'}), #Box_Tree
  ee.Feature(ee.Geometry.Point([-75.652248, 37.709249]), {'site_id': 'Channel_Point'}), #Channel_Point
  ee.Feature(ee.Geometry.Point([-75.942331, 37.175055]), {'site_id': 'Cushmans'}), #Cushmans
  ee.Feature(ee.Geometry.Point([-75.940586, 37.167409]), {'site_id': 'Gator'}), #Gator
  ee.Feature(ee.Geometry.Point([-75.814954, 37.485033]), {'site_id': 'Greens_Creek'}), #Greens_Creek
  ee.Feature(ee.Geometry.Point([-75.672871, 37.451132]), {'site_id': 'Hog'}), #Hog
  ee.Feature(ee.Geometry.Point([-75.900996, 37.346190]), {'site_id': 'Indiantown'}), #Indiantown
  ee.Feature(ee.Geometry.Point([-75.558264, 37.784154]), {'site_id': 'Kegotank'}), #Kegotank
  ee.Feature(ee.Geometry.Point([-75.861733, 37.416783]), {'site_id': 'Mill_Creek'}), #Mill_Creek
  ee.Feature(ee.Geometry.Point([-75.822928, 37.468043]), {'site_id': 'N_Brownsville'}), #N_Brownsville
  ee.Feature(ee.Geometry.Point([-75.929133, 37.287549]), {'site_id': 'Oyster'}), #Oyster
  ee.Feature(ee.Geometry.Point([-75.940690, 37.181277]), {'site_id': 'Steelmans'}), #Steelmans
  ee.Feature(ee.Geometry.Point([-75.447000, 37.880000]), {'site_id': 'Wallops'}), #Wallops
  ee.Feature(ee.Geometry.Point([-75.819360, 37.481998]), {'site_id': 'Woodland_Farm'}) #Woodland_Farm
])


In [100]:
#landsat 5 surface reflectance images

roi = ee.Geometry.Point(-75.876094, 37.395746)

filteredCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(1999, 2017,'year')) \
    .filter(ee.Filter.calendarRange(8, 9,'month')) \
    .filterBounds(plot_pts) \
    .filterMetadata('CLOUD_COVER', 'less_than', 100) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(filteredCollectionLS5.size().getInfo())
image0 = filteredCollectionLS5.first()
#print(image0.getInfo())

37


In [19]:
#get list of image ids in filtered collection
filteredCollectionLS5.aggregate_array('system:id').getInfo()

#figure out image dates!!

['LANDSAT/LT05/C01/T1_SR/LT05_014034_20060801',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20070804',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20070905',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20110831',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20030926',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20060918',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040912',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20040827',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20090825',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20020907',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20000816',
 'LANDSAT/LT05/C01/T1_SR/LT05_014034_201

In [96]:
#add image number for each of the images in the filtered collection - closer to 8/15 is best
image00 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20000901') #pushed to later date bc clouds
image01 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20010803') #clouds over some points
image02 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20020822') #CLOUDS - make composite??? how
image03 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20030825') #clouds?
image04 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20040811')
image05 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20050814')
image06 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20060817') #image0 is 2006 - lower cloud coverage, maybe use?
image07 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20070804')
image08 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20080822')
image09 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20090809')
image10 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20100828') #CLOUDS?
image11 = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_014034_20110815')

#image12 = ee.Image()
#image13 = ee.Image()
#landsat 8?

#determining cloud cover
cloudiness = image06.get('CLOUD_COVER')
print('CLOUD_COVER: ', cloudiness.getInfo())
props = geemap.image_props(image06)
props.getInfo()


CLOUD_COVER:  24


{'CLOUD_COVER': 24,
 'CLOUD_COVER_LAND': 11,
 'EARTH_SUN_DISTANCE': 1.012384,
 'ESPA_VERSION': '2_19_0c',
 'GEOMETRIC_RMSE_MODEL': 5.246,
 'GEOMETRIC_RMSE_MODEL_X': 2.992,
 'GEOMETRIC_RMSE_MODEL_Y': 4.309,
 'IMAGE_DATE': '2006-08-17',
 'IMAGE_QUALITY': 9,
 'LANDSAT_ID': 'LT05_L1TP_014034_20060817_20160910_01_T1',
 'LEVEL1_PRODUCTION_DATE': 1473520194000,
 'NOMINAL_SCALE': 30,
 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.5.0',
 'SATELLITE': 'LANDSAT_5',
 'SENSING_TIME': '2006-08-17T15:34:06.9000810Z',
 'SOLAR_AZIMUTH_ANGLE': 133.230423,
 'SOLAR_ZENITH_ANGLE': 31.816601,
 'SR_APP_VERSION': 'LEDAPS_3.2.0',
 'WRS_PATH': 14,
 'WRS_ROW': 34,
 'system:asset_size': '288.941016 MB',
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'sr_atmos_opacity',
  'sr_cloud_qa',
  'pixel_qa',
  'radsat_qa'],
 'system:id': 'LANDSAT/LT05/C01/T1_SR/LT05_014034_20060817',
 'system:index': 'LT05_014034_20060817',
 'system:time_end': '2006-08-17 15:34:06',
 'system:time_start': '2006-0

In [ ]:
#Cloud masking images 01-11

In [79]:
#Mapping
#Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(image07, vis_param, "Landsat5SR")

Map.centerObject(plot_pts, 10)
Map.set_plot_options(add_marker_cluster = True) #not sure what this does

#calculate ndvi
ndvi = image00.normalizedDifference(['B4', 'B3']).rename('NDVI')
vis_ndvi = {'min': -1, 
             'max': 1, 
             'palette': ['blue', 'white', 'green']}
#Map.addLayer(ndvi, vis_ndvi, "NDVI")

#add feature collection points for each plot
Map.addLayer(plot_pts)

Map

Map(center=[37.485005614872954, -75.76448856851104], controls=(WidgetControl(options=['position'], widget=HBox…

In [11]:
#extracting raster values for points; 
#from: https://developers.google.com/earth-engine/tutorials/community/extract-raster-values-for-points
#buffering points


In [70]:
#Exporting pixel values - any way to do this without manually selecting 
in_fc = plot_pts

#export data
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ndvipoints00.csv')
#out_shp = os.path.join(out_dir, 'points.shp') #shapefile exporting
geemap.extract_values_to_points(in_fc, image00, out_csv)

The image must be an instance of ee.Image.


In [118]:

# This example demonstrates the use of the pixel QA band to mask
# clouds in surface reflectance (SR) data.  It is suitable
# for use with any of the Landsat SR datasets.

# Function to cloud mask from the pixel_qa band of Landsat 8 SR data.
def maskL5sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  # Get the pixel QA band.
  qa = image.select('pixel_qa')

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  # Return the masked image, scaled to reflectance, without the QA bands.
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

# Map the function over image collection
collection = filteredCollectionLS5 \
    .map(maskL5sr)

composite = collection.median()

# Display the results.
Map.addLayer(composite, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "cloudfree composite")


In [129]:
masked00 = ee.Image(maskL5sr(image00))
Map.addLayer(masked00, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "2000 masked") #isn't perfect but good
Map.addLayer(image00, vis_param, "2000 unmasked")
Map.addLayer(collection.first(), {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.3}, "masked???")


In [128]:
print("hi")

hi
